# Imports

In [20]:
!pip install sypy tqdm dill reportlab # uncomment if the imports don't work
# !sudo apt update
# !sudo apt install texlive-latex-base

ERROR: Ignored the following versions that require a different python version: 0.0.1 Requires-Python >=3.12; 0.0.2 Requires-Python >=3.12
ERROR: Could not find a version that satisfies the requirement sypy (from versions: none)
ERROR: No matching distribution found for sypy


In [21]:
import os, sys
import sympy              # for symbolic modelling
from sympy import symbols, Matrix, Eq, simplify, solve, latex, denom, numer, sqrt, degree, init_printing, pprint, Poly
from tqdm import tqdm     # to create progress bars
from itertools import product


# # optional imports
# import dill               # to save/load the results
# # from google.colab import files
# from multiprocessing import Pool, Value, cpu_count, Manager
# from reportlab.lib.pagesizes import letter
# from reportlab.pdfgen import canvas

# Create the directory if it doesn't exist
print(f"# of cores: {os.cpu_count()}\nOS Name: {sys.platform}\nWorking Directory: {os.getcwd()}") # is 96 for TPU v2-8
init_printing()
# # symbols? #use this to find documentation on any object/functionts

# of cores: 2
OS Name: linux
Working Directory: /content


# Algorithms Definitions

## Set up global variables

In [22]:
# Define symbolic variables
s = symbols('s')
R1, R2, R3, R4, R5, RL, Rs      = symbols('R1 R2 R3 R4 R5 RL Rs')
C1, C2, C3, C4, C5, CL          = symbols('C1 C2 C3 C4 C5 CL')
L1, L2, L3, L4, L5, LL          = symbols('L1 L2 L3 L4 L5 LL')
Z1 , Z2 , Z3 , Z4 , Z5 , ZL, Zs = symbols('Z1 Z2 Z3 Z4 Z5 ZL Zs')

# Get symbolic variables (CG)
Iip, Iin, I1a, I1b, I2a, I2b = symbols('Iip Iin I1a I1b I2a I2b')
Vin, V2a, V2b, V1a, V1b, Va, Vb, Von, Vop, Vip, Vx = symbols('Vin V2a V2b V1a V1b Va Vb Von Vop Vip Vx')

inf = sympy.oo # infinity symbol in SymPy

# Transmission matrix coefficients
gm, ro, Cgd, Cgs    = symbols('gm ro Cgd Cgs')
a11, a12, a21, a22  = symbols('a11 a12 a21 a22')

T_select ={
    "simple" : Matrix([[0, -1/gm],[0, 0]]),
    "symbolic" : Matrix([[a11, a12],[a21, a22]]),
    "some_parasitic" : Matrix([[-1/(gm*ro), -1/gm],[0, 0]]),
    "full_parasitic" : Matrix([[(1/ro + s*Cgd)/(s*Cgd - gm), 1/(s*Cgd - gm)],[(Cgd*Cgs*ro*s + Cgd*gm*ro + Cgs + Cgd)*s/(s*Cgd - gm), (Cgs+Cgd)*s/(s*Cgd - gm)]])
}

# Select a transimision matrix
T_type  = "symbolic"
T_a     = T_select[T_type]
T_b     = T_select[T_type]

print(f"Transmission matrices are type: {T_type}")
print(T_a)
print(T_b)

Transmission matrices are type: symbolic
Matrix([[a11, a12], [a21, a22]])
Matrix([[a11, a12], [a21, a22]])


## Algorithm 1: Solve for TF
Algorithm 1 steps
1. Define impedance combinations
2. Define nodal equation
3. Loop through all impedance combinations and find H(s)

### 1. Define Z Combination

In [23]:
# (1) Define impedance combinations
def getCommonGateImpedanceCombinations():
    # Define impedance arrays (Based on Table 1)
    Zz1 = [R1]                      # R

    Zz2 = [R2,                                            # R
           1/(s*C2),                                      # C
           R2/(1 + R2*C2*s),                              # R || C
           R2 + 1/(C2*s),                                 # R + C
           s*L2 + 1/(s*C2),                               # L + C
           R2 + s*L2 + 1/(s*C2),                          # R + L + C
           R2 + (s*L2/(1 + L2*C2*s**2)),                  # R + (L || C)
           R2*(s*L2 + 1/(s*C2))/(R2 + (s*L2 + 1/(s*C2)))  # R2 || (L2 + C2)
           ]

    Zz3 = [R3,                                            # R
           s*L3,                                          # L
           1/(s*C3),                                      # C
           R3/(1 + R3*C3*s),                              # R || C
           R3 + 1/(C3*s),                                 # R + C
           (s*L3 + 1/(s*C3)),                             # L + C
           (L3*s)/(1 + L3*C3*s**2),                       # L || C
           R3 + s*L3 + 1/(s*C3),                          # R + L + C
           (1/R3 + s*C3+ 1/(s*L3))**-1,                   # R || L || C
           R3 + (s*L3/(1 + L3*C3*s**2)),                  # R + (L || C)
           R3*(s*L3 + 1/(s*C3))/(R3 + (s*L3 + 1/(s*C3)))  # R || (L + C)
           ]


    Zz4 = [R4,                                            # R
           s*L4,                                          # L
           1/(s*C4),                                      # C
           R4/(1 + R4*C4*s),                              # R || C
           R4 + 1/(C4*s),                                 # R + C
           (s*L4 + 1/(s*C4)),                             # L + C
           (L4*s)/(1 + L4*C4*s**2),                       # L || C
           R4 + s*L4 + 1/(s*C4),                          # R + L + C
           (1/R4 + s*C4+ 1/(s*L4))**-1,                   # R || L || C
           R4 + (s*L4/(1 + L4*C4*s**2)),                  # R + (L || C)
           R4*(s*L4 + 1/(s*C4))/(R4 + (s*L4 + 1/(s*C4)))  # R || (L + C)
           ]

    Zz5 = [R4,                                            # R
           s*L4,                                          # L
           1/(s*C4),                                      # C
           R4/(1 + R4*C4*s),                              # R || C
           R4 + 1/(C4*s),                                 # R + C
           (s*L4 + 1/(s*C4)),                             # L + C
           (L4*s)/(1 + L4*C4*s**2),                       # L || C
           R4 + s*L4 + 1/(s*C4),                          # R + L + C
           (1/R4 + s*C4+ 1/(s*L4))**-1,                   # R || L || C
           R4 + (s*L4/(1 + L4*C4*s**2)),                  # R + (L || C)
           R4*(s*L4 + 1/(s*C4))/(R4 + (s*L4 + 1/(s*C4)))  # R || (L + C)
           ]

    ZzL = [RL,                                            # R
           s*LL,                                          # L
           1/(s*CL),                                      # C
           RL/(1 + RL*CL*s),                              # R || C
           RL + 1/(CL*s),                                 # R + C
           (s*LL + 1/(s*CL)),                             # L + C
           (LL*s)/(1 + LL*CL*s**2),                       # L || C
           RL + s*LL + 1/(s*CL),                          # R + L + C
           (1/RL + s*CL+ 1/(s*LL))**-1,                   # R || L || C
           RL + (s*LL/(1 + LL*CL*s**2)),                  # R + (L || C)
           RL*(s*LL + 1/(s*CL))/(RL + (s*LL + 1/(s*CL)))  # R || (L + C)
           ]

    # Combine Z
    return {
       "all"         : product(Zz1, Zz2, Zz3, Zz4, Zz5, ZzL),
       "Z3_ZL"       : product([inf], [inf], Zz3, [inf], [inf], ZzL),
       "Z5_ZL"       : product([inf], [inf], [inf], [inf], Zz5, ZzL),
       "Z2_Z4_ZL"    : product([inf], Zz2, [inf], Zz4, [inf], ZzL),
       "Z2_Z5_ZL"    : product([inf], Zz2, [inf], [inf], Zz5, ZzL),
       "Z3_Z5_ZL"    : product([inf], [inf], Zz3, [inf], Zz5, ZzL)
    }

combos = getCommonGateImpedanceCombinations()
combos

{'all': <itertools.product at 0x7ccd44e21a80>,
 'Z3_ZL': <itertools.product at 0x7ccd44ff6a40>,
 'Z5_ZL': <itertools.product at 0x7ccd44ff5d40>,
 'Z2_Z4_ZL': <itertools.product at 0x7ccd44f8b3c0>,
 'Z2_Z5_ZL': <itertools.product at 0x7ccd44e36040>,
 'Z3_Z5_ZL': <itertools.product at 0x7ccd44e36080>}

### 2. Define Nodal Equations

In [24]:
# (2) Define Nodal Equations
def getCommonGateEquation(TransmissionMatrices, Z_arr = symbols('Z1 Z2 Z3 Z4 Z5 ZL')):
    # Get symbolic variables (CG)
    Z1 , Z2 , Z3 , Z4 , Z5 , ZL = Z_arr
    T_a, T_b = TransmissionMatrices

    # Define nodal equations (Eq. 4a-4h) -> list[ Eq(left-hand-side, right-hand-side), ... ]
    equations = [
         # 4a
         Eq(0, (Iip + I1a + I2a + (0 - Vip)/Z1 + (Vop - Vip)/Z2 + (Von - Vip)/Z5)),
         # 4b
         Eq(0, (Iin + I1b + I2b + (0 - Vin)/Z1 + (Von - Vin)/Z2 + (Vop - Vin)/Z5)),
         # 4c
         Eq(I2a, ((Vip - Vop)/Z2 + (Vx - Vop)/Z3 + (Von - Vop)/Z4 + (Vin - Vop)/Z5 + (0 - Vop)/ZL)),
         # 4d
         Eq(I2b, ((Vin - Von)/Z2 + (Vx - Von)/Z3 + (Vop - Von)/Z4 + (Vip - Von)/Z5 + (0 - Von)/ZL )),
         # unranked equation in the paper (between 4d and 4e)
         Eq(I1a, ((Vop - Vx)/Z3 + (Von - Vx)/Z3 - I1b)),
         # 4e
         Eq(Vop, Vip + V2a),
         Eq(Von, Vin + V2b),
         # 4f
         Eq(Vip, Vx - V1a),
         Eq(Vin, Vx - V1b),
         # 3g
         Eq(V1a, T_a[0,0]*V2a - T_a[0,1]*I2a),
         Eq(V1b, T_b[0,0]*V2b - T_b[0,1]*I2b),
         # 3h
         Eq(I1a, T_a[1,0]*V2a + T_a[1,1]*I2a),
         Eq(I1b, T_b[1,0]*V2b + T_b[1,1]*I2b)
     ]

    return equations

print(f"Common gate equation count: {len(getCommonGateEquation(TransmissionMatrices=[T_a, T_b]))}")

Common gate equation count: 13


### 3. Get Transfer Functions

In [25]:
# (3 - CG) Get transfer functions (TFs)
def _getCommonGateBaseHs(numerator = [Vop, Von], denominator = [Vip, Vin], Z_arr = symbols("Z1 Z2 Z3 Z4 Z5 ZL")):
    print(f"====CommonGate==== Z_arr = {Z_arr}")
    Z1, Z2, Z3, Z4, Z5, ZL = Z_arr
    TransmissioMatrices = [T_a, T_b]

    oPos, oNeg = numerator
    iPos, iNeg = denominator
    # Define nodal equations
    equations = getCommonGateEquation(TransmissioMatrices, Z_arr)
    print("(1) set up the nodal equation")

    # Solve for generic transfer function
    solution = solve(equations, [
        Vin, V2a, V2b, V1a, V1b, Va, Vb, Von, Vop, Vip, Vx,
        Iip, Iin, I1a, I1b, I2a, I2b
    ])
    print("(2) solved the base transfer function")

    if solution:
        print("FOUND THE BASE TF")
        baseHs = (solution[oPos] - solution[oNeg]) / (solution[iPos] - solution[iNeg])
        baseHs = simplify((baseHs.factor()))
        return baseHs

    return None


def getCommonGateTFs(impedanceBatch, baseHs):

    TFs = []
    if baseHs:
        for zCombo in tqdm(impedanceBatch, desc="Getting the TFs (CG)", unit="combo"):
            Z1, Z2, Z3, Z4, Z5, ZL = zCombo

            sub_dict = {symbols("Z1") : Z1,
                        symbols("Z2") : Z2,
                        symbols("Z3") : Z3,
                        symbols("Z4") : Z4,
                        symbols("Z5") : Z5,
                        symbols("ZL") : ZL}

            Hs = baseHs.subs(sub_dict)
            Hs = simplify((Hs.factor()))
            TFs.append(Hs)

    return TFs

# _getCommonGateBaseHs()

## Algorithm 2: Computing w0, Q, Qzz, K_LP, K_HP, K_BP
IN -> H(s), out -> temp = 0 if valid values (else temp = 1, meaning all values are invalid since filter total order is out of bound)

In [26]:
def _computeBiQuadParameters(tf): # HELPER FUNCTION
    s = symbols('s')

    # Determine numerator and denominator of H(s)
    dn = denom(tf)  # Denominator
    nm = numer(tf)  # Numerator
    DenOrder = degree(dn, s)
    NumOrder = degree(nm, s)

    # Coefficients of denominator
    a2 = dn.coeff(s, 2)  # Coefficient of s^2
    a1 = dn.coeff(s, 1)  # Coefficient of s^1
    a0 = dn.coeff(s, 0)  # Coefficient of s^0

    # Validate filter order and coefficients
    if not a2 or not a1 or not a0 or NumOrder > 2:
        return {
            'valid' : False,
            'message': "Invalid "}

    # Compute Q, w, Bandwidth
    wo = sqrt(simplify(a0 / a2))
    Q = simplify((a2 / a1) * wo)
    bandwidth = wo / Q

    # Coefficients of numerator
    b2 = nm.coeff(s, 2)  # Coefficient of s^2
    b1 = nm.coeff(s, 1)  # Coefficient of s^1
    b0 = nm.coeff(s, 0)  # Coefficient of s^0

    # Calculate filter constants
    # numerator(s) = b2 s^2 + b1 s^1 + b0
    # denom    (s) = a2 s^2 + a1 s^1 + a0
    if b2!=0:
        wz = sqrt(simplify(b0/b2))
        Qz = simplify((b2/b1) * wz) if b1 != 0 else None
    else:
        wz = None
        Qz = None

    K_HP = simplify(b2 / a2)
    K_BP = simplify(b1 / (a2 * bandwidth))
    K_LP = simplify(b0 / (a2 * wo**2))


    # Return results
    return {
        "valid" : True,
        "Q": Q,
        "wo": wo,
        "Bandwidth": bandwidth,
        "K_LP": K_LP,
        "K_HP": K_HP,
        "K_BP": K_BP,
        "Qz": Qz,
        "wz": wz
    }


def computeBiQuadParameters(TFs, impedanceCombinations):
    """
    Compute filter parameters for a list of transfer functions and impedance combinations.

    Parameters:
    - TFs: List of transfer functions.
    - impedanceCombinations: List of corresponding impedance combinations.

    Returns:
    - List of dictionaries containing impedance combination, transfer function, and parameters.
    """
    output = []
    count = 0
    # Wrap the zip iterator with tqdm for progress tracking
    for tf, impedanceCombo in tqdm(zip(TFs, impedanceCombinations),
                                    total=len(TFs),
                                    desc="Computing filter parameters",
                                    unit="filter"):
        parameters = _computeBiQuadParameters(tf)
        if parameters['valid']:
            output.append({"Impedance": impedanceCombo, "TF": tf, "Parameters": parameters})
            count += 1
        else:
            output.append({"Impedance": impedanceCombo, "TF": tf, "Parameters": None})
    return output

# Test Compute filter parameters
# s = symbols("s")
# tfs = [(s**2 +1 )/(3*s**2 + 4*s + 4)]
# output = computeBiQuadParameters(tfs, [1])
# print()
# print("++++++++")
# print(output)
# output[0]["TF"]

## Algorithm 3: Determine Filter Type
* IN -> Parameters from Algorithm 2: w0, Q, Qz, K_LP, K_HP, K_BP
* OUT -> Filter type classification (e.g., HP, BP, LP, BS, GE)





In [27]:
def getFilterType(parameters):
    """
    Determine the type of filter based on K_LP, K_HP, K_BP, and Qz.

    Parameters:
    - parameters: dict containing valid, Q, wo, K_LP, K_HP, K_BP, and Qz.

    Returns:
    - List of filter types and associated parameters.
    """
    valid = parameters.get("valid", False)
    K_LP = parameters.get("K_LP", 0)
    K_HP = parameters.get("K_HP", 0)
    K_BP = parameters.get("K_BP", 0)
    Q = parameters.get("Q")
    wo = parameters.get("wo")
    Qz = parameters.get("Qz")

    # Result containers
    filter_results = {
        "HP": None,
        "BP": None,
        "LP": None,
        "BS": None,
        "GE": None
    }

    if not valid:
        return filter_results

    # Classification based on filter constants
    if K_BP == 0 and K_LP == 0:
        filter_results["HP"] = {"wo": wo, "Q": Q, "K_HP": K_HP}

    if K_HP == 0 and K_LP == 0:
        filter_results["BP"] = {"wo": wo, "Q": Q, "K_BP": K_BP}

    if K_HP == 0 and K_BP == 0:
        filter_results["LP"] = {"wo": wo, "Q": Q, "K_LP": K_LP}

    if K_BP == 0 and K_HP != 0 and K_LP != 0:
        filter_results["BS"] = {"wo": wo, "Q": Q, "K_HP": K_HP, "K_LP": K_LP}

    if K_LP == K_HP and K_BP != 0 and K_LP != 0:
        if Qz is None:
            Qz = simplify(K_LP * Q / K_BP)
        filter_results["GE"] = {"wo": wo, "Q": Q, "Qz": Qz, "K_LP": K_LP}

    return filter_results

def classifyFilter(filterParameter_results):
    """
    Classify transfer functions into filter types.

    Parameters:
    - tf_results: List of transfer function results from `computeFilterParameters`.

    Returns:
    - Filter classifications.
    """
    classifiedResults = []
    count = 0
    for result in filterParameter_results:
        parameters = result.get("Parameters")
        if parameters:
            filter_types = getFilterType(parameters)
            classifiedResults.append({
                "Impedance": result["Impedance"],
                "TF": result["TF"],
                "Filter Types": filter_types
            })
            count += 1
        else:
            classifiedResults.append({
                "Impedance": result["Impedance"],
                "TF": result["TF"],
                "Filter Types": None
            })
    return classifiedResults

# # Test Compute filter parameters
# s = symbols("s")

# tfs = [(s**2 - 1)/(3*s**2 + 4*s + 4), (2*s + 1)/(s**2 + 1)]
# output = computeBiQuadParameters(tfs, [1, 2])

# print()
# print("++++++++")
# print(output)
# print("++++++++")

# classification = classifyFilter(output)
# # print(classification)

# summary, counts = summarizeFilterType(classification)
# counts

## Helper Functions

### Filtering and Processing

In [28]:
def findFilterInClassification(classifications, filterType, printResult=False):
    output = []
    count = 0

    for entity in classifications:
        # print(f"entity : {entity}")
        if (entity["Filter Types"]) and (entity["Filter Types"][filterType]):
            output.append({
                           "Impedance" : entity["Impedance"],
                           "TF": entity["TF"],
                           "Parameters" : entity["Filter Types"][filterType]
                           })
            count += 1

    if printResult:
        print(f"{filterType} : {len(output)}")
    return output, count

def summarizeFilterType(classifications, filterTypes=["HP", "BP", "LP", "BS", "GE"],printResult=False):
    output = {}
    counts = {}
    for fType in filterTypes:
        output[fType], counts[fType] = findFilterInClassification(classifications, fType, printResult)
    return output, counts


### Printing

In [29]:
def save_TFs_as_pdf(TFs, output_filename):
    """
    Saves a list of transfer functions as rendered mathematical expressions to a PDF.

    Parameters:
    - TFs: List of Sympy expressions representing transfer functions.
    - output_filename: Name of the output PDF file.
    """
    output_filename = "Outputs/" + output_filename + ".pdf"
    os.makedirs("Outputs", exist_ok=True)

    # Prepare the PDF canvas
    c = canvas.Canvas(output_filename, pagesize=letter)
    width, height = letter
    margin = 50  # Margin around the page
    y_position = height - margin  # Starting position for writing

    for i, tf in enumerate(TFs, 1):
        # Render the transfer function as an image
        image_filename = f"Outputs/TF_{i}.png"
        tf_latex = f"H(s) = {latex(tf)}"
        sympy.preview(
            tf_latex,
            output='png',
            viewer='file',
            filename=image_filename,
            euler=False,  # Use standard fonts instead of Euler fonts
        )

        # Check if there's enough space for the next image; if not, start a new page
        image_height = 100  # Adjust if needed for image size
        if y_position - image_height < margin:
            c.showPage()  # Add a new page
            y_position = height - margin

        # Draw the image on the PDF
        c.drawImage(image_filename, margin, y_position - image_height, width=width - 2 * margin, height=image_height)

        # Move the y_position for the next item
        y_position -= image_height + 20  # Add spacing between entries

    c.save()
    print(f"PDF saved to {output_filename}")

    # Cleanup: Remove temporary image files
    for i in range(1, len(TFs) + 1):
        os.remove(f"Outputs/TF_{i}.png")


def save_TFs_as_LaTex(TFs, output_filename):
    """
        # !sudo apt update
        # !sudo apt install texlive-latex-base

    # # Compile the LaTeX file into a PDF (make sure pdflatex is installed)
    # !pdflatex output_filename.tex

    # Optional: download the PDF
    # files.download("transfer_functions.pdf")

    # Create a Latex file to report the transfer functions
    """

    header = r"""
    \documentclass{article}
    \usepackage{amsmath}
    \usepackage{geometry}
    \geometry{landscape, a1paper, margin=1in}  % Adjust paper size and margins
    \begin{document}
    \section*{Derived Transfer Functions}
    """

    footer = r"\end{document}"

    # LaTeX filename
    output_filename = "Outputs/" + output_filename + ".tex"
    os.makedirs("Outputs", exist_ok=True)

    # Write the LaTeX code into the file
    with open(output_filename, "w") as latex_file:
        latex_file.write(header)
        for i, tf in enumerate(TFs, 1):
            latex_file.write(f"\\subsection*{{Transfer Function {i}}}\n")
            latex_file.write(f"\\[ H(s) = {latex(tf)} \\]\n")
            # # Add a page break after each equation (optional)
            # latex_file.write("\\newpage\n")
        latex_file.write(footer)


# Experiment Run - Common Gate
Select from the below defintions
T_select =
- "simple" : [[0, -1/gm],[0, 0],
- "symbolic" : [[a11, a12],[a21, a22],
- "some_parasitic" : [[-1/(gm*ro), -1/gm],[0, 0],
- "full_parasitic" : [[(1/ro + s*Cgd)/(s*Cgd - gm), 1/(s*Cgd - gm)],[(Cgd*Cgs*ro*s + Cgd*gm*ro + Cgs + Cgd)*s/(s*Cgd - gm), (Cgs+Cgd)*s/(s*Cgd - gm)]])


In [30]:
T_type = "simple"
T_a = T_select[T_type]
T_b = T_select[T_type]

numerator_Hs   = [Vop, Von]
denominator_Hs = [Vip, Vin]

# Re-creating Table II
baseHs_CG = _getCommonGateBaseHs(numerator_Hs, denominator_Hs)

# oo is the symbol for infitity (Table II)
# ------------------------------------------------------------
baseHs_CG_Z3ZL      = sympy.limit(baseHs_CG, Z1, inf)         # Z1, Z2, Z4, and Z5 are set to oo (infinity)
baseHs_CG_Z3ZL      = sympy.limit(baseHs_CG_Z3ZL, Z2, inf)
baseHs_CG_Z3ZL      = sympy.limit(baseHs_CG_Z3ZL, Z4, inf)
baseHs_CG_Z3ZL      = sympy.limit(baseHs_CG_Z3ZL, Z5, inf)
# ------------------------------------------------------------
baseHs_CG_Z5ZL      = sympy.limit(baseHs_CG, Z1, inf)         # Z1, Z2, Z3, and Z4 are set to oo (infinity)
baseHs_CG_Z5ZL      = sympy.limit(baseHs_CG_Z5ZL, Z2, inf)
baseHs_CG_Z5ZL      = sympy.limit(baseHs_CG_Z5ZL, Z3, inf)
baseHs_CG_Z5ZL      = sympy.limit(baseHs_CG_Z5ZL, Z4, inf)
# ------------------------------------------------------------
baseHs_CG_Z2Z4ZL    = sympy.limit(baseHs_CG, Z1, inf)         # Z1, Z3 and Z5 are set to oo (infinity)
baseHs_CG_Z2Z4ZL    = sympy.limit(baseHs_CG_Z2Z4ZL, Z3, inf)
baseHs_CG_Z2Z4ZL    = sympy.limit(baseHs_CG_Z2Z4ZL, Z5, inf)
# ------------------------------------------------------------
baseHs_CG_Z2Z5ZL    = sympy.limit(baseHs_CG, Z1, inf)         # Z1, Z3, and Z4 are set to oo (infinity)
baseHs_CG_Z2Z5ZL    = sympy.limit(baseHs_CG_Z2Z5ZL, Z3, inf)
baseHs_CG_Z2Z5ZL    = sympy.limit(baseHs_CG_Z2Z5ZL, Z4, inf)
# ------------------------------------------------------------
baseHs_CG_Z3Z5ZL    = sympy.limit(baseHs_CG, Z1, inf)         # Z1, Z2, and Z4 are set to oo (infinity)
baseHs_CG_Z3Z5ZL    = sympy.limit(baseHs_CG_Z3Z5ZL, Z2, inf)
baseHs_CG_Z3Z5ZL    = sympy.limit(baseHs_CG_Z3Z5ZL, Z4, inf)
# ------------------------------------------------------------

baseHs = {
    "all"       : baseHs_CG,
    "Z3_ZL"     : baseHs_CG_Z3ZL,
    "Z5_ZL"     : baseHs_CG_Z5ZL,
    "Z2_Z4_ZL"  : baseHs_CG_Z2Z4ZL,
    "Z2_Z5_ZL"  : baseHs_CG_Z2Z5ZL,
    "Z3_Z5_ZL"  : baseHs_CG_Z3Z5ZL
}

baseHs["Z3_Z5_ZL"]

====CommonGate==== Z_arr = (Z1, Z2, Z3, Z4, Z5, ZL)
(1) set up the nodal equation
(2) solved the base transfer function
FOUND THE BASE TF


  Z₃⋅ZL⋅(Z₅⋅gm - 1)  
─────────────────────
Z₃⋅Z₅ + Z₃⋅ZL + Z₅⋅ZL

In [43]:
select = "Z2_Z5_ZL"
combos = getCommonGateImpedanceCombinations()
batch = list(combos[select])
tf = baseHs[select]

TFs = getCommonGateTFs(batch, tf)

# Output summary of results
print("Number of transfer functions found: {}".format(len(TFs)))
for i, tf in enumerate(TFs, 1):
    print("H(s) {}: {}".format(i, tf))

# save_TFs_as_LaTex(TFs, "CG_testRun")

Getting the TFs (CG): 100%|██████████| 968/968 [07:57<00:00,  2.03combo/s]


Number of transfer functions found: 968
H(s) 1: RL*(R2*R4*gm - R2 + R4)/(R2*R4 + R2*RL + R4*RL)
H(s) 2: LL*s*(R2*R4*gm - R2 + R4)/(LL*R2*s + LL*R4*s + R2*R4)
H(s) 3: (R2*R4*gm - R2 + R4)/(CL*R2*R4*s + R2 + R4)
H(s) 4: RL*(R2*R4*gm - R2 + R4)/(CL*R2*R4*RL*s + R2*R4 + R2*RL + R4*RL)
H(s) 5: (CL*RL*s + 1)*(R2*R4*gm - R2 + R4)/(CL*R2*R4*s + CL*R2*RL*s + CL*R4*RL*s + R2 + R4)
H(s) 6: (CL*LL*s**2 + 1)*(R2*R4*gm - R2 + R4)/(CL*LL*R2*s**2 + CL*LL*R4*s**2 + CL*R2*R4*s + R2 + R4)
H(s) 7: LL*s*(R2*R4*gm - R2 + R4)/(CL*LL*R2*R4*s**2 + LL*R2*s + LL*R4*s + R2*R4)
H(s) 8: (CL*LL*s**2 + CL*RL*s + 1)*(R2*R4*gm - R2 + R4)/(CL*LL*R2*s**2 + CL*LL*R4*s**2 + CL*R2*R4*s + CL*R2*RL*s + CL*R4*RL*s + R2 + R4)
H(s) 9: LL*RL*s*(R2*R4*gm - R2 + R4)/(CL*LL*R2*R4*RL*s**2 + LL*R2*R4*s + LL*R2*RL*s + LL*R4*RL*s + R2*R4*RL)
H(s) 10: (R2*R4*gm - R2 + R4)*(CL*LL*RL*s**2 + LL*s + RL)/(CL*LL*R2*R4*s**2 + CL*LL*R2*RL*s**2 + CL*LL*R4*RL*s**2 + LL*R2*s + LL*R4*s + R2*R4 + R2*RL + R4*RL)
H(s) 11: RL*(CL*LL*s**2 + 1)*(R2*R4*gm 

In [44]:
# Preview the TF
TFs[0]

RL⋅(R₂⋅R₄⋅gm - R₂ + R₄)
───────────────────────
 R₂⋅R₄ + R₂⋅RL + R₄⋅RL 

In [45]:
filterParameters = computeBiQuadParameters(TFs, batch)
classifications = classifyFilter(filterParameters)
filterTypes = ["HP", "BP", "LP", "BS", "GE"]

summary, counts = summarizeFilterType(classifications, filterTypes, printResult=True)

Computing filter parameters: 100%|██████████| 968/968 [00:24<00:00, 39.37filter/s]

HP : 74
BP : 100
LP : 74
BS : 0
GE : 0


In [42]:
filterParameters[0]

{'Impedance': (oo, oo, oo, oo, R4, RL),
 'TF': RL*(R4*gm - 1)/(R4 + RL),
 'Parameters': None}

In [47]:
summary["BP"][0]

{'Impedance': (oo, R2, oo, oo, R4, LL*s + 1/(CL*s)),
 'TF': (CL*LL*s**2 + 1)*(R2*R4*gm - R2 + R4)/(CL*LL*R2*s**2 + CL*LL*R4*s**2 + CL*R2*R4*s + R2 + R4),
 'Parameters': {'wo': sqrt(1/(CL*LL)),
  'Q': LL*sqrt(1/(CL*LL))*(R2 + R4)/(R2*R4),
  'K_BP': 0}}

In [49]:
summary["BP"][0]['TF']

      ⎛       2    ⎞                            
      ⎝CL⋅LL⋅s  + 1⎠⋅(R₂⋅R₄⋅gm - R₂ + R₄)       
────────────────────────────────────────────────
          2             2                       
CL⋅LL⋅R₂⋅s  + CL⋅LL⋅R₄⋅s  + CL⋅R₂⋅R₄⋅s + R₂ + R₄

In [18]:
# !pdflatex Outputs/CS_testRun.tex

# Saving the environment parameters
Use the code block below to save the experiment.

**Requires the Dill library to be pip installed**

In [ ]:
# Saving the variable with dill
variables = {'TFs': TFs, 'Impedance_combinations': impedance_combinations}

with open('sympy_variables.dill', 'wb') as f:
    dill.dump(variables, f)

In [ ]:
# Loading the variabels with dill
with open('sympy_variables.dill', 'rb') as f:
    loaded_variables = dill.load(f)

# Access the loaded symbolic expressions and variables
TFs = loaded_variables['TFs']
Impedance_combinations = loaded_variables['Impedance_combinations']

# Archive

In [ ]:
from itertools import product

array1 = [1, 2]
array2 = ['a', 'b']

combinations = product(array1, array2)
print(list(combinations))
# Output: [(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]


[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]


In [ ]:

def getCommonGateTFs_cache(Zz_arr,baseHs):
    TFs = [] # an array of dictionaries {}"TF": tf, "combo": {"Z1": z1, "Z2": z2 ... "ZL":zl}}
    counter = 0
    total_combinations = 1

    for z in Zz_arr:
        total_combinations *= len(z)

    for z1 in Zz_arr[0]:
        Hs1 = baseHs.subs({symbols("Z1") : z1})
        # Hs1 = simplify(Hs1.factor())
        for z2 in Zz_arr[1]:
            Hs2 = Hs1.subs({symbols("Z2") : z2})
            # Hs2 = simplify(Hs2.factor())
            for z3 in Zz_arr[2]:
                Hs3 = Hs2.subs({symbols("Z3") : z3})
                # Hs3 = simplify(Hs3.factor())
                for z4 in Zz_arr[3]:
                    Hs4 = Hs3.subs({symbols("Z4") : z4})
                    # Hs4 = simplify(Hs4.factor())
                    for z5 in Zz_arr[4]:
                        Hs5 = Hs4.subs({symbols("Z5") : z5})
                        # Hs5 = simplify(Hs5.factor())
                        for zL in Zz_arr[5]:
                            HsFinal = Hs5.subs({symbols("ZL") : zL})
                            HsFinal = simplify(HsFinal.factor())
                            TFs.append({
                                "Z_arr" : {
                                    "Z1" : z1,
                                    "Z2" : z2,
                                    "Z3" : z3,
                                    "Z4" : z4,
                                    "Z5" : z5,
                                    "ZL" : zL,
                                },
                                "TF" : HsFinal
                            })
                            # Increment and display the counter
                            counter += 1
                            if counter % 25 == 0:
                                print(f"Processed {counter}/{total_combinations} combinations", end="\r")
    return TFs



In [ ]:
from sympy import symbols, oo, limit

# Define the variable and expression
x = symbols('x')
y = symbols('y')

expr = (x**3 + 1) / (2*x**2 - y)

# Compute the limit as x approaches infinity
result = limit(expr, x, oo)

print(f"expr : {expr}")

print(f"Limit as x -> oo: {result}")
